In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from scipy import special
import math

In [3]:
pbp17 = pd.read_csv('Events_2017.csv')

In [4]:
team = pbp17[((pbp17['WTeamID']==1157) | (pbp17['LTeamID']==1157)) & ((pbp17['EventType']=='made1_free') | 
             (pbp17['EventType']=='miss1_free') | (pbp17['EventType']=='made2_dunk') | (pbp17['EventType']=='miss2_dunk') | 
             (pbp17['EventType']=='made2_tip') | (pbp17['EventType']=='miss2_tip') | (pbp17['EventType']=='made2_lay') | 
             (pbp17['EventType']=='miss2_lay') | (pbp17['EventType']=='made2_jump') | (pbp17['EventType']=='miss2_jump') | 
             (pbp17['EventType']=='made3_jump') | (pbp17['EventType']=='miss3_jump') | (pbp17['EventType']=='turnover'))]

In [5]:
def get_team_data(teamID):
    team = pbp17[((pbp17['WTeamID']==teamID) | (pbp17['LTeamID']==teamID)) & ((pbp17['EventType']=='made1_free') | 
             (pbp17['EventType']=='miss1_free') | (pbp17['EventType']=='made2_dunk') | (pbp17['EventType']=='miss2_dunk') | 
             (pbp17['EventType']=='made2_tip') | (pbp17['EventType']=='miss2_tip') | (pbp17['EventType']=='made2_lay') | 
             (pbp17['EventType']=='miss2_lay') | (pbp17['EventType']=='made2_jump') | (pbp17['EventType']=='miss2_jump') | 
             (pbp17['EventType']=='made3_jump') | (pbp17['EventType']=='miss3_jump') | (pbp17['EventType']=='turnover'))]
    return team

In [6]:
get_team_data(1157)

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
2,20434642,2017,11,1104,1157,0,0,15,1157,638145,turnover
4,20434644,2017,11,1104,1157,3,0,26,1104,637420,made3_jump
5,20434645,2017,11,1104,1157,0,0,63,1157,638145,miss2_jump
7,20434647,2017,11,1104,1157,0,0,68,1104,637410,miss3_jump
11,20434651,2017,11,1104,1157,0,0,82,1157,638155,turnover
13,20434653,2017,11,1104,1157,0,0,121,1104,637423,turnover
14,20434654,2017,11,1104,1157,0,0,136,1157,638156,turnover
16,20434656,2017,11,1104,1157,0,0,146,1104,637410,miss3_jump
19,20434659,2017,11,1104,1157,0,0,165,1157,638156,miss2_jump
26,20434666,2017,11,1104,1157,4,0,177,1104,637413,made1_free


In [7]:
detailed = pd.read_csv('RegularSeasonDetailedResults.csv')
stats18 = detailed[detailed['Season']==2018]
stats18

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
76636,2018,11,1104,82,1272,70,N,0,26,57,...,17,22,36,19,24,7,17,7,3,22
76637,2018,11,1107,69,1233,67,H,0,24,62,...,22,11,14,7,20,14,7,6,3,21
76638,2018,11,1112,101,1319,67,H,0,34,57,...,13,17,30,12,11,11,13,3,1,24
76639,2018,11,1113,94,1226,74,H,0,34,65,...,24,13,22,8,21,17,14,6,2,20
76640,2018,11,1116,95,1359,56,H,0,36,63,...,29,8,15,15,17,12,19,6,1,20
76641,2018,11,1120,102,1313,74,H,0,35,63,...,34,20,27,9,20,15,16,11,3,26
76642,2018,11,1124,107,1146,66,H,0,39,59,...,20,10,15,7,14,15,12,4,4,19
76643,2018,11,1127,76,1288,59,H,0,26,57,...,15,11,15,14,25,9,18,10,2,20
76644,2018,11,1130,85,1263,65,H,0,33,68,...,32,12,21,8,24,13,11,1,2,20
76645,2018,11,1132,78,1180,69,A,0,26,54,...,20,19,26,11,24,7,19,5,4,24


In [9]:
def get_percentages(teamID):
    team_data = stats18[(stats18['WTeamID'] == teamID) | (stats18['LTeamID'] == teamID)]
    made_two=0
    made_three=0
    made_one=0
    miss_one = 0
    miss_two = 0
    miss_three = 0
    turnovers = 0
    
    for i,row in team_data.iterrows():
        if row['WTeamID'] == teamID:
            made_two += row['WFGM']
            made_one += row['WFTM']
            made_three += row['WFGM3']
            turnovers += row['WTO']
            miss_one += (row['WFTA'] - row['WFTM'])
            miss_two += (row['WFGA'] - row['WFGM'])
            miss_three += (row['WFGA3'] - row['WFGM3'])
        elif row['LTeamID'] == teamID:
            made_two += row['LFGM']
            made_one += row['LFTM']
            made_three += row['LFGM3']
            turnovers += row['LTO']
            miss_one += (row['LFTA'] - row['LFTM'])
            miss_two += (row['LFGA'] - row['LFGM'])
            miss_three += (row['LFGA3'] - row['LFGM3'])
    possessions = made_one + miss_one + made_two + miss_two + made_three + miss_three + turnovers
    misses = miss_one + miss_two + miss_three + turnovers
    arr = [made_one/possessions, miss_one/possessions, made_two/possessions, miss_two/possessions, made_three/possessions, 
           miss_three/possessions, turnovers/possessions, misses/possessions]
    return arr

print(get_percentages(1402))
print(get_percentages(1157))

[0.12331701346389229, 0.059057527539779685, 0.2141982864137087, 0.2876376988984088, 0.06181150550795594, 0.1251529987760098, 0.1288249694002448, 0.6006731946144431]
[0.13889734998477002, 0.05239110569600975, 0.2116966189460859, 0.2817544928419129, 0.06000609198903442, 0.12031678342978983, 0.1349375571123972, 0.5893999390801097]


In [14]:
team1_percs = get_percentages(1157)
team2_percs = get_percentages(1402)
seq_time = team1_time + team2_time
team1_miss = team1_percs[7]
team2_miss = team2_percs[7]
make1_miss = team1_percs[0] * team2_miss
make2_miss = team1_percs[3] * team2_miss
make3_make = team1_percs[5] * team2_miss
miss_make1 = team2_percs[0] * team1_miss
miss_make2 = team2_percs[3] * team1_miss
miss_miss = team1_miss * team2_miss
miss_make3 = team2_percs[5] * team1_miss
make1_make1 = team1_percs[0] * team2_percs[0]
make1_make2 = team1_percs[0] * team2_percs[2]
make1_make3 = team1_percs[0] * team2_percs[4]
make2_make1 = team1_percs[2] * team2_percs[0]
make2_make2 = team1_percs[2] * team2_percs[2]
make2_make3 = team1_percs[2] * team2_percs[4]
make3_make1 = team1_percs[4] * team2_percs[0]
make3_make2 = team1_percs[4] * team2_percs[2]
make3_make3 = team1_percs[4] * team2_percs[4]            
            

In [46]:
denom = []
def breakdown(num):
    denom = []
    for a in range(0,num+1):
        for b in range(0,num+1):
            for c in range(0,num+1):
                if num == c + 2*b + 3*a:
                    denom.append([c,b,a])    
    return denom
breakdown(10)

[[10, 0, 0],
 [8, 1, 0],
 [6, 2, 0],
 [4, 3, 0],
 [2, 4, 0],
 [0, 5, 0],
 [7, 0, 1],
 [5, 1, 1],
 [3, 2, 1],
 [1, 3, 1],
 [4, 0, 2],
 [2, 1, 2],
 [0, 2, 2],
 [1, 0, 3]]

In [50]:
def calc_prob(k, denom, team):
    ones = denom[0]
    twos = denom[1]
    threes = denom[2]
    misses = k - sum(denom)
    choose = (special.comb(k,threes) * special.comb(k-threes,twos) * special.comb(k-threes-twos,ones))
    prob = team[4]**(threes) * team[2]**(twos) * team[0]**(ones) * team1_miss**(misses)
    return prob * choose

In [112]:
def win_prob(team1, team2, score1,score2, start_time, end_time):
    team1_time = possession_data(team1)[1]
    team2_time = possession_data(team2)[1]
    seq_time = team1_time + team2_time
    score_difference = score1 - score2
    remainder = min(score1,score2)
    total_sum = 0
    event_sum = 0
    possessions = int(math.ceil((end_time - start_time) / seq_time))
    matrix = np.zeros((101-remainder,101-remainder))
    team1_arr = np.zeros((101-remainder))
    team2_arr = np.zeros((101-remainder))
        
    for i in range(0,101-remainder):
        k = possessions
        prob1 = []
        prob2 = []
        team_scores = breakdown(i)
        for denom in team_scores:
            p1 = calc_prob(k, denom, team1_percs)
            p2 = calc_prob(k, denom, team2_percs)
            prob1.append(p1)
            prob2.append(p2)
        team1_arr[i] = sum(prob1)
        team2_arr[i] = sum(prob2)
            
    for i in range(0,101-remainder):
        for j in range(0,101-remainder):
            both = team1_arr[i] * team2_arr[j]
            matrix[i][j] = both 
            total_sum += both
            if i > (j - score_difference):
                event_sum += matrix[i][j]
                
    return event_sum / total_sum
    



In [123]:
game_data = pbp17[(pbp17['WTeamID'] == 1157) & (pbp17['LTeamID'] == 1402)]
game_data

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
1071638,21506278,2017,63,1157,1402,0,0,21,1157,638152,assist
1071639,21506279,2017,63,1157,1402,3,0,21,1157,638150,made3_jump
1071640,21506280,2017,63,1157,1402,0,0,47,1157,638152,foul_pers
1071641,21506281,2017,63,1157,1402,0,0,47,1402,641877,miss1_free
1071642,21506282,2017,63,1157,1402,0,0,47,1157,638144,reb_def
1071643,21506283,2017,63,1157,1402,3,1,47,1402,641877,made1_free
1071644,21506284,2017,63,1157,1402,0,0,67,1157,638156,assist
1071645,21506285,2017,63,1157,1402,5,1,67,1157,638145,made2_lay
1071646,21506286,2017,63,1157,1402,0,0,89,1402,641879,miss3_jump
1071647,21506287,2017,63,1157,1402,0,0,89,1157,638145,reb_def


In [124]:
wp = pd.DataFrame(columns=['Win Probability'])
new_data = game_data.iloc[1:,:][(game_data['WPoints'] != 0) & (game_data['WPoints'] != 0)]
game_data = game_data.iloc[0:1,:].append(new_data)
game_data.index = range(0,len(game_data))
game_data

C:\Users\Adhithya\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
0,21506278,2017,63,1157,1402,0,0,21,1157,638152,assist
1,21506279,2017,63,1157,1402,3,0,21,1157,638150,made3_jump
2,21506283,2017,63,1157,1402,3,1,47,1402,641877,made1_free
3,21506285,2017,63,1157,1402,5,1,67,1157,638145,made2_lay
4,21506293,2017,63,1157,1402,7,1,148,1157,638152,made2_jump
5,21506294,2017,63,1157,1402,8,1,148,1157,638152,made1_free
6,21506296,2017,63,1157,1402,8,3,175,1402,641879,made2_lay
7,21506305,2017,63,1157,1402,10,3,237,1157,638145,made2_lay
8,21506321,2017,63,1157,1402,10,6,335,1402,641874,made3_jump
9,21506331,2017,63,1157,1402,10,8,413,1402,641881,made2_tip


In [125]:
index = 0
for i,row in game_data.iterrows():
    w_prob = win_prob(row['WTeamID'],row['LTeamID'],row['WPoints'],row['LPoints'],row['ElapsedSeconds'],2400)
    wp.loc[index] = w_prob
    index += 1

In [126]:
pd.concat([game_data, wp], axis=1)

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,Win Probability
0,21506278,2017,63,1157,1402,0,0,21,1157,638152,assist,0.474979
1,21506279,2017,63,1157,1402,3,0,21,1157,638150,made3_jump,0.580880
2,21506283,2017,63,1157,1402,3,1,47,1402,641877,made1_free,0.546325
3,21506285,2017,63,1157,1402,5,1,67,1157,638145,made2_lay,0.616277
4,21506293,2017,63,1157,1402,7,1,148,1157,638152,made2_jump,0.685582
5,21506294,2017,63,1157,1402,8,1,148,1157,638152,made1_free,0.717183
6,21506296,2017,63,1157,1402,8,3,175,1402,641879,made2_lay,0.653870
7,21506305,2017,63,1157,1402,10,3,237,1157,638145,made2_lay,0.722426
8,21506321,2017,63,1157,1402,10,6,335,1402,641874,made3_jump,0.623924
9,21506331,2017,63,1157,1402,10,8,413,1402,641881,made2_tip,0.551134
